In [1]:
%pip install openai
%pip install -U chromadb
%pip install sentence-transformers
%pip install langchain langchain-community langchain-experimental
%pip install numpy pandas tqdm
%pip install "pydantic<2.0"
%pip install openai==0.28.1


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.21
    Uninstalling pydantic-1.10.21:
      Successfully uninstalled pydantic-1.10.21
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chroma-migrate 0.0.7 requires duckdb==0.7.1, but you have duckdb 1.2.1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
  Using cached pydantic-1.10.21-cp312-cp312-win_amd64.whl.metadata (155 kB)
Using cached pydantic-1.10.21-cp312-cp312-win_amd64.whl (2.2 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.3
    Uninstalling pydantic-2.11.3:
      Successfully uninstalled pydantic-2.11.3
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chroma-migrate 0.0.7 requires duckdb==0.7.1, but you have duckdb 1.2.1 which is incompatible.
langchain 0.3.23 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.21 which is incompatible.
langchain-core 0.3.51 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.21 which is incompatible.
pydantic-settings 2.8.1 requires pydantic>=2.7.0, but you have pydantic 1.10.21 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.


In [2]:
key = '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'

# Preload the math strategy list (static from Capstone PDF)
math_strategies = """
CONTEXT 3 (Math-Specific Teaching Strategies):
- Allow use of fingers and scratch paper
- Use diagrams and draw math concepts
- Present activities involving visual, auditory, tactile, and kinesthetic modalities
- Offer manipulatives throughout instruction
- Use rhythm and music to teach math facts
- Practice new strategies repeatedly until mastered
- Use games and real-life applications to make math engaging
- Emphasize reverses (e.g., 2+3 = 3+2) to reinforce number sense
- Connect concepts like division and fractions to place value
"""

LLMs = {
    'llama':'meta-llama/llama-4-maverick:free',
    'gemini':'google/gemini-2.0-flash-exp:free',
    'deepseek':'deepseek/deepseek-r1-zero:free'
}

In [3]:
import os
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import openai
from chromadb import PersistentClient

# SETUP — your persistent DB + OpenRouter
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = key
llm_model = LLMs['llama']  # Update this to your actual model on OpenRouter

# Executive Skill Map
executive_skill_map = {
    "dyscalculia": ["Enhancing Working Memory", "Cultivating Metacognition", "Fostering Organization", "Promoting, Planning, and Prioritizing"],
    "dyslexia": ["Task Initiation", "Sustained Attention", "Metacognition", "Organization"],
    "autism": ["Emotional Control", "Flexibility", "Goal-Directed Persistence", "Time Management"]
}

# Load ChromaDB collections
client = PersistentClient(path="../chroma_store1/")

lesson_collection = client.get_or_create_collection("lesson_plans")
exec_collection = client.get_or_create_collection("exec_skills")

# FUNCTION: Generate the augmented lesson plan
def generate_adaptive_lesson_plan(grade, topic, subject, disorder):
    disorder_key = disorder.lower()
    exec_skills = executive_skill_map.get(disorder_key, [])
    # Get lesson chunks
    lesson_results = lesson_collection.query(
    query_texts=[f"{topic} in {subject} for grade {grade}"],  # semantic hint
    n_results=5,
    where={
        "$and": [
            {"grade": str(grade).strip()},
            {"subject": subject.strip()},
            {"topic": topic.strip()}
        ]
    },
    include=["documents", "metadatas"]
)
    lesson_context = "\n\n".join(lesson_results["documents"][0])
    # print(f'lesson_context:{lesson_context}')

    # Get exec strategy chunks
    exec_contexts = []
    for skill in exec_skills:
        results = exec_collection.query(
            query_texts=[f"Strategies for {skill}"],
            n_results=2,
            where={"executive_skill": skill},
            include=["documents"]
        )
        exec_contexts.extend(results["documents"][0])
    exec_context = "\n\n".join(exec_contexts)

    #print(f'exec_context:{exec_context}')

    # Prompt template
    prompt = f"""
You are a lesson planning assistant for special education teachers. Your task is to generate a complete math lesson plan that aligns with the lesson structure provided, while incorporating cognitive strategies from executive function skills helpful for students with {disorder.title()}.

Use the academic lesson content provided in CONTEXT 1 and the executive functioning strategies provided in CONTEXT 2. Match the exact structure and tone of the uploaded lesson plans.

CONTEXT 1 (Lesson Plan Content):
{lesson_context}

CONTEXT 2 (Executive Function Strategies for {disorder.title()}):
{exec_context}
{math_strategies}

---

Please write the lesson using the following structure:

1. Introduction (15 min)
2. Multi-Sensory Exploration (20 min)
3. Concept Practice (15–20 min)
4. Patterns / Deeper Understanding (optional)
5. Real-Life Applications (15 min)
6. Wrap-Up & Reflection (10 min)

Each section should include:
- Method
- Activities
- Executive Function Strategy
"""

    # LLM call to OpenRouter
    response = openai.ChatCompletion.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": "You are a supportive and creative educational assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content


In [4]:
collections = client.list_collections()
print([col.name for col in collections])

['exec_skills', 'lesson_plans']


In [5]:
print(lesson_collection.peek())

{'ids': ['lesson0_intro_context', 'lesson0_instructional_steps', 'lesson0_assessment', 'lesson0_differentiation', 'lesson1_intro_context', 'lesson1_instructional_steps', 'lesson1_assessment', 'lesson1_differentiation', 'lesson2_intro_context', 'lesson2_instructional_steps'], 'embeddings': array([[-0.06508687,  0.02420383,  0.00731116, ...,  0.0157884 ,
        -0.01527833,  0.05818786],
       [-0.06440627,  0.09550655, -0.01461828, ...,  0.08317908,
        -0.02853062,  0.02799912],
       [-0.04580441,  0.00480579, -0.01315066, ...,  0.01375909,
         0.01020204, -0.02798666],
       ...,
       [-0.00085461,  0.0562727 , -0.01412269, ...,  0.10347889,
        -0.03711525,  0.08102922],
       [-0.03787833,  0.02596228,  0.0413523 , ...,  0.04186512,
         0.01997391,  0.07512492],
       [-0.03696867,  0.00385485,  0.07021764, ...,  0.04068062,
        -0.0194161 ,  0.0756363 ]]), 'documents': ['Algebra I-Equations and Inequalities-A.4ae - Progressing Through Equations.pdf --

In [6]:
subject = 'Number and Number Sense'
topic = 'Reading, writing, and identifying the place value of six-digit numerals'
grade = '3'

In [7]:
lesson_results = lesson_collection.query(
    query_texts=[f"{topic} in {subject} for grade {grade}"],  # semantic hint
    n_results=5,
    where={
        "$and": [
            {"grade": str(grade).strip()},
            {"subject": subject.strip()},
            {"topic": topic.strip()}
        ]
    },
    include=["documents", "metadatas"]
)

lesson_context = "\n\n".join(lesson_results["documents"][0])
print(f'lesson_context:{lesson_context}')

lesson_context:Grade 3-Number and Number Sense-3.1a - Place Value.pdf ---
Place Value
Strand:
Number and Number Sense
Topic:
Reading, writing, and identifying the place value of six-digit numerals
Primary SOL:
3.1 The student will
a) read, write, and identify the place and value of each digit in a six-
digit whole number, with and without models
Materials
 Classroom set of base-10 blocks
 Place-value Mats (attached)
 Digit Cards (attached)
 Plastic baggies
 Construction paper in six different colors
 Adding machine tape or sentence strips
 Scissors
 Stapler
Vocabulary
Value, digit, place value, number, numeral, standard form, expanded form, compare

 Questions
o Which digit holds the highest value in a six-digit number?
o What do you do when you do not have any tens in a number?
o What do you do when you do not have any hundred thousands in a number? How
many digits would be in your number?
o Starting with the ones place, what are the place-value spaces in order up to the
hund

In [8]:
disorder = 'dyscalculia'
exec_contexts = []
disorder_key = disorder.lower()
exec_skills = executive_skill_map.get(disorder_key, [])
print(exec_skills)
for skill in exec_skills:
    results = exec_collection.query(
        query_texts=[f"Strategies for {skill}"],
        n_results=2,
        where={"executive_skill": skill},
        include=["documents"]
    )
    exec_contexts.extend(results["documents"][0])
    print(exec_contexts)
exec_context = "\n\n".join(exec_contexts)

print(f'exec_context:{exec_context}')

['Enhancing Working Memory', 'Cultivating Metacognition', 'Fostering Organization', 'Promoting, Planning, and Prioritizing']
['Enhancing Working Memory Working memory is the capacity to hold information in mind while performing complex tasks. We rely on working memory all the time. It’s the ability to run out to the store to buy a few things and remember what they are without having to write them down. When you remember to stop by the dry cleaner on your way home from work, you’re using working memory. When you look up a phone number in the phone book and remember it long enough to make the call, you’re using working memory. When your spouse asks you to do something and you say, “I’ll do it as soon as I finish loading the dishwasher,” and then you actually remember to do it, chances are your working memory is pretty good. Odds are it’s not so good, however, if you can’t remember anyone’s birthday, you tend to return home with only half your errands done unless you have a written agenda

In [12]:
print(generate_adaptive_lesson_plan(grade, topic, subject, "dyscalculia"))


**Lesson Plan: Exploring Place Value**

**Grade Level:** 3

**Subject:** Math

**Topic:** Reading, writing, and identifying the place value of six-digit numerals

### 1. Introduction (15 min)

* **Method:** Direct Instruction with Visual Aids
* **Activities:**
	1. Review the concept of place value up to the thousands place using base-10 blocks.
	2. Introduce the concept of place value up to the hundred-thousand place using a place-value mat and a six-digit number example (e.g., 165,724).
	3. Discuss the value of each digit within the number.
* **Executive Function Strategy:** Enhance Working Memory by using visual aids and breaking down complex information into smaller chunks.

### 2. Multi-Sensory Exploration (20 min)

* **Method:** Guided Practice with Manipulatives
* **Activities:**
	1. Divide students into groups of six and distribute place-value headbands, a set of 24 Digit Cards, and a Place-value Mat.
	2. Have each group create six-digit numbers by randomly selecting a card and 